In [ ]:
import os
import pandas as pd
from SignalProcessor import SignalProcessor

pd.options.plotting.backend = 'plotly'

repo_dir = '/home/cam/Documents/database_tools/'
os.chdir(repo_dir)

worker = SignalProcessor(
    files=['physionet.org/files/mimic3wdb/1.0/30/3000480/3000480_0018'],
    win_len=1024,
    fs=125,
)

windows, sim, snr, hr = worker.run(
    sim1=0.6,
    sim2=0.9,
    snr_t=20,
    hr_diff=1/6,
    f0_low=0.667,
    f0_high=3.0
)
len(windows)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from Preprocessing.SignalLevelFiltering import get_snr

i = 1000
x, y = windows[i][0], windows[i][1]

scaler = MinMaxScaler(feature_range=(60, 120))
x = scaler.fit_transform(x.reshape(-1, 1)).reshape(-1,)

df = pd.DataFrame(
    dict(
        pleth=x,
        abp=y,
    )
)

df.plot()

In [ ]:
# trace1 = go.Scatter(
#     x=np.arange(0, len(x), 1),
#     y=x,
#     name='PLETH',
#     marker=dict(color='rgb(34,163,192)'),
# )

# trace2 = go.Scatter(
#     x=idx_plotx,
#     y=x[idx_plotx],
#     name='Minima',
#     mode='markers',
# )

# fig = make_subplots(rows=1, cols=1)
# fig.add_trace(trace1, row=1, col=1)
# fig.add_trace(trace2, row=1, col=1)

# fig.show()

# trace1 = go.Scatter(
#     x=np.arange(0, len(x), 1),
#     y=y,
#     name='PLETH',
#     marker=dict(color='rgb(34,163,192)'),
# )

# trace2 = go.Scatter(
#     x=idx_plot,
#     y=y[idx_plot],
#     name='Minima',
#     mode='markers',
# )

# fig = make_subplots(rows=1, cols=1)
# fig.add_trace(trace1, row=1, col=1)
# fig.add_trace(trace2, row=1, col=1)

# fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from neurokit2.ppg import ppg_findpeaks
from heartpy.preprocessing import flip_signal

def segment_beats(x, fs=125):
    x_flip = flip_signal(x)
    idx = ppg_findpeaks(x_flip, sampling_rate=fs)['PPG_Peaks']
    return idx

i = 1
x, y = windows[i][0], windows[i][1]

idx_plotx = segment_beats(x)
idx_x = np.concatenate(
    [
        np.array(idx_plotx[0:-1]),
        np.array(idx_plotx[1:]),
    ]
).reshape(-1, 2, order='F')

idx_plot = segment_beats(y)
idx_y = np.concatenate(
    [
        np.array(idx_plot[0:-1]),
        np.array(idx_plot[1:]),
    ]
).reshape(-1, 2, order='F')

In [ ]:
len(idx_x)

In [ ]:
len(idx_y)

In [ ]:
def get_similarity(x, y):
    """
    Calculates time or spectral similarity.

    Args:
        x (np.ndarray): PLETH data.
        y (np.ndarray): ABP data.
        spectral (boolean): If True calculate fft
            of signals.

    Returns:
        coef (float): Pearson correlation coefficient.
    """
    diff = len(x) - len(y)
    if diff < 0:
        y = y[0:diff]
    elif diff > 0:
        x = x[0:-1 * diff]


    x_bar = np.mean(x)
    y_bar = np.mean(y)
    y_temp = y - y_bar
    x_temp = x - x_bar
    covar = np.sum( (x_temp * y_temp) )
    var = np.sqrt( np.sum( (x_temp ** 2 ) ) * np.sum( (y_temp ** 2) ) )
    coef = covar / var
    return coef

sim = []
last_beat = x[0:idx_x[0, 0]]
for i, j in idx_x:
    beat = x[i:j]
    sim.append(
        get_similarity(x=last_beat,
                       y=beat)
    )
    last_beat = beat
sim

In [ ]:
sim = []
last_beat = y[0:idx_y[0, 0]]
for i, j in idx_y:
    beat = y[i:j]
    sim.append(
        get_similarity(x=last_beat,
                       y=beat)
    )
    last_beat = beat
sim